# 概述

***Pandas的字符串处理：***
* 使用方法：先获取Series的str属性，然后在属性上调用函数；
* 只能在字符串列上使用，不能数字列上使用
* DataFrame上没有str属性和处理方法
* Series.str并不是python原生字符串，而是自己的一套方法，不过大部分和原生str很相似

# 读取数据

In [1]:
import pandas as pd

In [2]:
fPath = './datas/tianqi.csv'

In [3]:
df = pd.read_csv(fPath)

In [4]:
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel
0,2018-01-01,3℃,-6℃,晴~多云,东北风,1-2级,59,良,2
1,2018-01-02,2℃,-5℃,阴~多云,东北风,1-2级,49,优,1
2,2018-01-03,2℃,-5℃,多云,北风,1-2级,28,优,1
3,2018-01-04,0℃,-8℃,阴,东北风,1-2级,28,优,1
4,2018-01-05,3℃,-6℃,多云~晴,西北风,1-2级,50,优,1


In [5]:
df.dtypes

ymd          object
bWendu       object
yWendu       object
tianqi       object
fengxiang    object
fengli       object
aqi           int64
aqiInfo      object
aqiLevel      int64
dtype: object

# 获取Series的str属性，使用各种字符串处理函数

In [6]:
df['bWendu'].str

In [10]:
df['bWendu'].str.replace('℃','')

0       3
1       2
2       2
3       0
4       3
       ..
360    -5
361    -3
362    -3
363    -2
364    -2
Name: bWendu, Length: 365, dtype: object

In [8]:
# 是否是数字
df['bWendu'].str.isnumeric()

0      False
1      False
2      False
3      False
4      False
       ...  
360    False
361    False
362    False
363    False
364    False
Name: bWendu, Length: 365, dtype: bool

In [12]:
# df['aqi'].str.len()

# 使用str的startwith、contains得到bool的Series可以做条件查询

In [13]:
condition = df['ymd'].str.startswith('2018-03')

In [14]:
condition

0      False
1      False
2      False
3      False
4      False
       ...  
360    False
361    False
362    False
363    False
364    False
Name: ymd, Length: 365, dtype: bool

In [15]:
df[condition].head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel
59,2018-03-01,8℃,-3℃,多云,西南风,1-2级,46,优,1
60,2018-03-02,9℃,-1℃,晴~多云,北风,1-2级,95,良,2
61,2018-03-03,13℃,3℃,多云~阴,北风,1-2级,214,重度污染,5
62,2018-03-04,7℃,-2℃,阴~多云,东南风,1-2级,144,轻度污染,3
63,2018-03-05,8℃,-3℃,晴,南风,1-2级,94,良,2


# 需要多次str处理的链式操作
提取2018-03的月份，将日期处理为20180301这样的形式，提取月份字符串201803

In [17]:
df['ymd'].str.replace('-','')

0      20180101
1      20180102
2      20180103
3      20180104
4      20180105
         ...   
360    20181227
361    20181228
362    20181229
363    20181230
364    20181231
Name: ymd, Length: 365, dtype: object

In [20]:
# 每次调用函数，都返回一个新的Series
df['ymd'].str.replace('-','').str.slice(0,6)

0      201801
1      201801
2      201801
3      201801
4      201801
        ...  
360    201812
361    201812
362    201812
363    201812
364    201812
Name: ymd, Length: 365, dtype: object

In [22]:
# slice就是切片语法，可以直接用
df['ymd'].str.replace('-','').str[0:6]

0      201801
1      201801
2      201801
3      201801
4      201801
        ...  
360    201812
361    201812
362    201812
363    201812
364    201812
Name: ymd, Length: 365, dtype: object

# 使用正则表达式的处理

In [23]:
# 添加新列
def get_Ymd_cn(x):
    year,month,day = x['ymd'].split('-')
    return f"{year}年{month}月{day}日"

df['中文日期'] = df.apply(get_Ymd_cn,axis=1)

In [24]:
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,中文日期
0,2018-01-01,3℃,-6℃,晴~多云,东北风,1-2级,59,良,2,2018年01月01日
1,2018-01-02,2℃,-5℃,阴~多云,东北风,1-2级,49,优,1,2018年01月02日
2,2018-01-03,2℃,-5℃,多云,北风,1-2级,28,优,1,2018年01月03日
3,2018-01-04,0℃,-8℃,阴,东北风,1-2级,28,优,1,2018年01月04日
4,2018-01-05,3℃,-6℃,多云~晴,西北风,1-2级,50,优,1,2018年01月05日


问题：怎么将’2018年12月31日‘中的年、月、日三个中文字符去除？

In [25]:
# 方法1：链式replace
df['中文日期'].str.replace('年','').str.replace('月','').str.replace('日','')

0      20180101
1      20180102
2      20180103
3      20180104
4      20180105
         ...   
360    20181227
361    20181228
362    20181229
363    20181230
364    20181231
Name: 中文日期, Length: 365, dtype: object

Series.str 默认开启了正则表达式模式

In [28]:
# 方法2：正则表达式替换
df['中文日期'].str.replace('[年月日]','')

0      20180101
1      20180102
2      20180103
3      20180104
4      20180105
         ...   
360    20181227
361    20181228
362    20181229
363    20181230
364    20181231
Name: 中文日期, Length: 365, dtype: object